# Measurement Grouping

Since current quantum hardware is limited to single-qubit projective measurement, only terms commuting within individual qubit's subspace can be measured together. These terms are said to be qubit-wise commuting (QWC). 

Thus, one can not measure the entire electronic Hamiltonian $\hat H$ at once, and instead needs to separate it into fragments. 
$$\hat H = \sum_n \hat H_n$$
where each $\hat H_n$ is a QWC fragment. 

In [1]:
from utility import * 

/Users/ritabanchowdhury/.local/lib/python3.7/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


Here we use $H_2$ as an example for finding QWC fragments. Notice below that each fragment has the same terms on all qubits.  

To show differences between QWC and more advanced grouping, we didn't use the qubit-tappering techinique shown in step 2.

In [2]:
lih = get_qubit_hamiltonian(mol='lih', geometry=1.5, basis='sto3g', qubit_transf='jw')

qwc_list = get_qwc_group(lih)
print('Fragments 1: \n{}\n'.format(qwc_list[4]))
print('Fragments 2:\n{}\n'.format(qwc_list[1]))
print('Number of fragments: {}'.format(len(qwc_list)))

Fragments 1: 
-0.00018431338897360644 [X0 Z1 Z2 Y3 Y4 Z5 Z6 Z7 Z8 Z9 Z10 X11] +
0.06558452315458405 [Z6 Z8] +
0.06980180803300676 [Z6 Z9] +
0.06980180803300676 [Z7 Z8] +
0.06558452315458405 [Z7 Z9]

Fragments 2:
-0.0014982042400604148 [Y0 Z1 Z2 Z3 Z4 Z5 Y6 Y7 Z8 Z9 Z10 Y11] +
0.09907704108249506 [Z1 Z8] +
0.09662223425385628 [Z1 Z9] +
0.06278876343471385 [Z2 Z8] +
0.06878552428444606 [Z2 Z9] +
0.06878552428444606 [Z3 Z8] +
0.06278876343471385 [Z3 Z9] +
0.060225501399544155 [Z4 Z8] +
0.07055432072184806 [Z4 Z9] +
0.07055432072184806 [Z5 Z8] +
0.060225501399544155 [Z5 Z9] +
-0.2287824778363614 [Z8] +
0.07823637778985217 [Z8 Z9] +
0.06245512523136902 [Z8 Z10] +
-0.22878247783636138 [Z9] +
0.06732342777645647 [Z9 Z10]

Number of fragments: 154


By applying extra unitaries, one may rotate more terms of $\hat H$ into a QWC fragment.  

Recall that in digital quantum computing, the expectation value of $\hat H_n$ given a trial wavefunction $|\psi\rangle$ is 
$$ E_n =\ \langle\psi| \hat H_n | \psi\rangle$$
Inserting unitary transformation $\hat U_n$ does not change the expectation value.
$$ E_n =\ \langle\psi| \hat U_n^\dagger \hat U_n \hat H_n \hat U_n^\dagger \hat U_n  |\psi\rangle$$ 
This nonetheless changes the trial wavefunction and the terms to be measured. 
$$ |\psi\rangle \rightarrow \hat U_n |\psi\rangle = |\phi\rangle$$
$$ \hat H_n \rightarrow \hat U_n \hat H_n \hat U_n^\dagger = \hat A_n$$
The transformation of $|\psi \rangle$ can be done on the quantum computer, and the transformation of $\hat H_n$ is possible on the classical computer. 

Now, although $\hat A_n$ needs to be a QWC fragment to be measurable on a quantum computer, $\hat H_n$ does not. 
Instead, if we restrict $\hat U_n$ to be a clifford operation, the terms in $\hat H$ need only mutually commute. 

Here, we obtain measurable parts of $H_2$ by partitioning its terms into mutually commuting fragments. 

In [3]:
comm_groups = get_commuting_group(lih)
print('Number of mutually commuting fragments: {}'.format(len(comm_groups)))
print('The first commuting group')
print(comm_groups[1])

Number of mutually commuting fragments: 36
The first commuting group
-4.103591882717407 [] +
0.0036744565021407344 [X0 Y1 Y2 X3] +
-5.845579096315396e-05 [X0 Z1 Z2 X3 Y4 Y5] +
-0.001894147985049468 [X0 Z1 Z2 X3 Y6 Y7] +
-0.0018941479850494677 [X0 Z1 Z2 X3 Y8 Y9] +
-0.00010502189355815098 [X0 Z1 Z2 X3 Y10 Y11] +
0.0036744565021407344 [Y0 X1 X2 Y3] +
-5.845579096315396e-05 [Y0 Z1 Z2 Y3 X4 X5] +
-0.001894147985049468 [Y0 Z1 Z2 Y3 X6 X7] +
-0.0018941479850494677 [Y0 Z1 Z2 Y3 X8 X9] +
-0.00010502189355815098 [Y0 Z1 Z2 Y3 X10 X11] +
0.0948664760225029 [Z0 Z3] +
-5.845579096315396e-05 [X1 X2 X4 X5] +
-0.001894147985049468 [X1 X2 X6 X7] +
-0.0018941479850494677 [X1 X2 X8 X9] +
-0.00010502189355815097 [X1 X2 X10 X11] +
-5.845579096315396e-05 [Y1 Y2 Y4 Y5] +
-0.001894147985049468 [Y1 Y2 Y6 Y7] +
-0.0018941479850494677 [Y1 Y2 Y8 Y9] +
-0.00010502189355815097 [Y1 Y2 Y10 Y11] +
0.0948664760225029 [Z1 Z2] +
-0.010328819322303916 [X4 X5 Y6 Y7] +
-0.01032881932230391 [X4 X5 Y8 Y9] +
-0.006575744899176

To see this fragment is indeed measurable, one can construct the corresponding unitary operator $\hat U_n$.

In [4]:
uqwc = get_qwc_unitary(comm_groups[1])
print('This is unitary, U * U^+ = I ')
print(uqwc * uqwc)

This is unitary, U * U^+ = I 
(0.9999999999999999+0j) []


Applying this unitary gives the qubit-wise commuting form of the first mutually commuting group

In [5]:
qwc = remove_complex(uqwc * comm_groups[1] * uqwc)
print(qwc)

-4.10359188271699 [] +
0.09486647602250285 [X0] +
5.8455790963148286e-05 [X0 X1 X2 Z3 Z5] +
0.0018941479850493493 [X0 X1 X2 Z3 Z7] +
0.0018941479850493493 [X0 X1 X2 Z3 Z9] +
0.00010502189355815406 [X0 X1 X2 Z3 Z11] +
-0.003674456502141037 [X0 Z3] +
0.09486647602250285 [X1 X2] +
-0.003674456502141037 [X1 X2 Z3] +
5.8455790963148286e-05 [X1 X2 Z3 X4 Z5] +
0.0018941479850493493 [X1 X2 Z3 X6 Z7] +
0.0018941479850493493 [X1 X2 Z3 X8 Z9] +
0.00010502189355815406 [X1 X2 Z3 X10 Z11] +
-5.8455790963148286e-05 [X1 X2 X4 Z5] +
-0.0018941479850493493 [X1 X2 X6 Z7] +
-0.0018941479850493493 [X1 X2 X8 Z9] +
-0.00010502189355815406 [X1 X2 X10 Z11] +
0.08470391802239871 [X4] +
0.010328819322303685 [X4 Z5 Z7] +
0.010328819322303683 [X4 Z5 Z9] +
0.0065757448991767995 [X4 Z5 Z11] +
-5.8455790963148286e-05 [Z5] +
0.010328819322303685 [Z5 X6 Z7] +
0.010328819322303683 [Z5 X8 Z9] +
0.0065757448991767995 [Z5 X10 Z11] +
0.0782363777898527 [X6] +
0.0042172848784222845 [X6 Z7 Z9] +
0.004868302545087672 [X6 Z7 Z1

In addition, current quantum computer can measure only the $z$ operators. Thus, QWC fragments with $x$ or $y$ operators require extra single-qubit unitaries that rotate them into $z$.  

In [6]:
uz = get_zform_unitary(qwc)
print("Checking whether U * U^+ is identity: {}".format(uz * uz))

allz = remove_complex(uz * qwc * uz)
print("\nThe all-z form of qwc fragment:\n{}".format(allz))

Checking whether U * U^+ is identity: 0.9999999999999998 []

The all-z form of qwc fragment:
-4.103591882716997 [] +
0.09486647602250284 [Z0] +
5.8455790963148116e-05 [Z0 Z1 Z2 Z3 Z5] +
0.0018941479850493534 [Z0 Z1 Z2 Z3 Z7] +
0.0018941479850493534 [Z0 Z1 Z2 Z3 Z9] +
0.0001050218935581536 [Z0 Z1 Z2 Z3 Z11] +
-0.00367445650214104 [Z0 Z3] +
0.09486647602250284 [Z1 Z2] +
-0.00367445650214104 [Z1 Z2 Z3] +
5.8455790963148116e-05 [Z1 Z2 Z3 Z4 Z5] +
0.0018941479850493534 [Z1 Z2 Z3 Z6 Z7] +
0.0018941479850493534 [Z1 Z2 Z3 Z8 Z9] +
0.0001050218935581536 [Z1 Z2 Z3 Z10 Z11] +
-5.8455790963148116e-05 [Z1 Z2 Z4 Z5] +
-0.0018941479850493534 [Z1 Z2 Z6 Z7] +
-0.0018941479850493534 [Z1 Z2 Z8 Z9] +
-0.0001050218935581536 [Z1 Z2 Z10 Z11] +
0.0847039180223987 [Z4] +
0.010328819322303673 [Z4 Z5 Z7] +
0.010328819322303671 [Z4 Z5 Z9] +
0.006575744899176785 [Z4 Z5 Z11] +
-5.8455790963148116e-05 [Z5] +
0.010328819322303673 [Z5 Z6 Z7] +
0.010328819322303671 [Z5 Z8 Z9] +
0.006575744899176785 [Z5 Z10 Z11] +
0.078